Establish connection

In [1]:
%reload_ext google.cloud.bigquery
import os
os.environ["GOOGLE_APPLICATION_CREDENTIALS"]="C:\\Users\\Cermati\\AppData\\Roaming\\gcloud\\application_default_credentials.json"
print(os.environ["GOOGLE_APPLICATION_CREDENTIALS"])

C:\Users\Cermati\AppData\Roaming\gcloud\application_default_credentials.json


Insert query here

In [16]:
%%bigquery param1 --project athena-179008
with tiket_new_score as (
select * from (
select orderId, finalScore, modelName,
row_number() OVER (PARTITION BY orderId ORDER BY createdAt DESC) AS rank
from data-platform-indodana.vayu.indodana_athena_application_scoring_results
where upper(modelName) = 'SCORING_CRITERIA_CLI_TIKET_20210331')
where rank = 1
),

identity_score_query as (
select app.orderId
,feat1.value as izi_mobile_phone_number_ages
,feat2.value as verified_izi_mobile_phone_whatsapp
,CAST(feat3.value as NUMERIC) as email_age_ever_name_match
,CAST(feat4.value as NUMERIC) as phone_age_ever_name_match
,CAST(feat5.value AS NUMERIC) as pefindo_max_phone_number_match
,CAST(feat6.value as NUMERIC) as hibp_name_match
from data-platform-indodana.vayu.indodana_athena_applications app
LEFT JOIN data-platform-indodana.vayu.indodana_featurestores_object_features_scores feat1 on app.orderId = feat1.object_id AND feat1.feature_slug_name = 'izi_mobile_phone_number_ages'
LEFT JOIN data-platform-indodana.vayu.indodana_featurestores_object_features_scores feat2 on app.orderId = feat2.object_id AND feat2.feature_slug_name = 'verified_izi_mobile_phone_whatsapp'
LEFT JOIN data-platform-indodana.vayu.indodana_featurestores_object_features_scores feat3 on app.orderId = feat3.object_id AND feat3.feature_slug_name = 'email_age_ever_name_match'
LEFT JOIN data-platform-indodana.vayu.indodana_featurestores_object_features_scores feat4 on app.orderId = feat4.object_id AND feat4.feature_slug_name = 'phone_age_ever_name_match'
LEFT JOIN data-platform-indodana.vayu.indodana_featurestores_object_features_scores feat5 on app.orderId = feat5.object_id AND feat5.feature_slug_name = 'pefindo_max_phone_number_match'
LEFT JOIN data-platform-indodana.vayu.indodana_featurestores_object_features_scores feat6 on app.orderId = feat6.object_id AND feat6.feature_slug_name = 'hibp_name_match'),
identity_score_mapping as (
select *, 
(460 
+ izi_mobile_phone_number_ages_score 
+ verified_izi_mobile_phone_whatsapp_score
+ email_age_ever_name_match_score 
+ phone_age_ever_name_match_score 
+ pefindo_max_phone_number_match_score 
+ hibp_name_match_score) as identity_score_manual
from (
select *
,CASE 
    WHEN izi_mobile_phone_number_ages IN ("12month+","10-12month") THEN 39
    WHEN izi_mobile_phone_number_ages IN ("8-10month","0-1month","1-2month","6-8month","3-4month","2-3month","5-6month","4-5month") THEN 2
    WHEN izi_mobile_phone_number_ages IN ("-99","-999") THEN -19
    ELSE 0 END AS izi_mobile_phone_number_ages_score
,CASE 
    WHEN verified_izi_mobile_phone_whatsapp = 'not verified' THEN 6
    WHEN verified_izi_mobile_phone_whatsapp = 'no' THEN -35
    WHEN verified_izi_mobile_phone_whatsapp = 'verified' THEN 63
    ELSE 0 END AS verified_izi_mobile_phone_whatsapp_score
,CASE 
    WHEN email_age_ever_name_match <= -99 THEN -20
    WHEN email_age_ever_name_match <= 5 THEN -11
    WHEN email_age_ever_name_match > 5 THEN 42
    ELSE NULL END AS email_age_ever_name_match_score
,CASE 
    WHEN phone_age_ever_name_match <= -99 THEN -11
    WHEN phone_age_ever_name_match <= 5 THEN 0
    WHEN phone_age_ever_name_match > 5 THEN 25
    ELSE NULL END AS phone_age_ever_name_match_score
,CASE 
    WHEN pefindo_max_phone_number_match <= -99 THEN -11
    WHEN pefindo_max_phone_number_match <= 0 THEN 0
    WHEN pefindo_max_phone_number_match > 0 THEN 82
    ELSE NULL END AS pefindo_max_phone_number_match_score
,CASE 
    WHEN hibp_name_match <= 0 THEN -9
    WHEN hibp_name_match > 0 THEN 31    
    ELSE NULL END AS hibp_name_match_score
from identity_score_query)
),

rank_data AS (
    SELECT idNumber,version, row_number() OVER (PARTITION BY idNumber
                                 ORDER BY version DESC) AS rank
    FROM data-platform-indodana.vayu.indodana_athena_pefindos
    ),

pefindos AS (
    SELECT a.*, rank, ind.identificationKTP,
    ind.mainAddressLine,
    ind.mainAddressCity,
    ind.mainAddressCountry,
    ind.mainAddressDistrict,
    ind.mainAddressPostalCode,
    ind.mainAddressStreet,
    ind.dateOfBirth,
    ind.contactMobilePhone,
    ind.employeeName,
    ind.employeeSector,
    ind.motherMaidenName,
    ind.fullName
    FROM  data-platform-indodana.vayu.indodana_athena_pefindos a
    LEFT JOIN (SELECT * FROM rank_data WHERE rank=1) b ON a.idNumber = b.idNumber AND a.version = b.version
    LEFT JOIN data-platform-indodana.vayu.indodana_athena_pefindo_individuals ind ON a.id=ind.pefindoId AND ind.version=a.version
    WHERE rank=1 ),

pefindos_agg_fetch AS (
select idNumber,count(*) as flag_pefindo
from pefindos
group by 1
),

pefindo_features AS
(SELECT
    *
FROM
    athena-179008.vayu_data_mart.dm_application_risk_feature_pefindo_version_before_submit
),

pefindo_classification AS
(SELECT 
    orderId,
    CASE 
        WHEN 
            ((pf.percentage_latest_dpd_15_1y >= 0)
             AND (pf.percentage_latest_active_dpd_3 = 0)
             AND (pf.count_written_off_unique_creditor = 0))
        THEN 
            TRUE 
        ELSE 
            FALSE 
    END AS okay,
    CASE
        WHEN 
            ((pf.percentage_good_credit_card_payments_1y >= 0.8)
            AND (pf.count_outstanding_amount_active_cc > 0)
            AND (pf.count_written_off_unique_creditor = 0))
        THEN
            TRUE
        ELSE
            FALSE
    END AS okay_cc,
    CASE
        WHEN
            ((pf.max_limit_active_cc >= 15000000)
            OR (pf.sum_limit_active_cc >= 30000000))
        THEN
            "High"
        WHEN
            ((pf.max_limit_active_cc >= 8000000)
            OR (pf.sum_limit_active_cc >= 20000000))
        THEN
            "Medium"
        WHEN
            ((pf.max_limit_active_cc > 0)
            AND (pf.sum_limit_active_cc > 0))
        THEN
            "Low"
        ELSE
            "Other"
    END AS active_limit_status,
    CASE
        WHEN 
            (pf.sum_paylater_settled_1y > 5000000)
        THEN
            TRUE
        ELSE
            FALSE
    END AS settled_paylater,
    CASE
        WHEN
            ((pf.sum_paylater_disbursed_1y > 10000000)
            AND (pf.sum_paylater_settled_1y > 1000000))
        THEN
            TRUE
        ELSE
            FALSE
    END AS high_paylater,
    pf.days_from_active_credit_card_disbursement_date,
    pf.percentage_outstanding_credit_card_active,
    pefindo_id_count
FROM
    pefindo_features pf
),

pefindo_rule AS
(SELECT
    orderId,
    CASE 
        WHEN pefindo_id_count IS NULL OR pefindo_id_count = 0 THEN "NO PEFINDO"
        WHEN
            ((active_limit_status = "High")
            AND (okay_cc = TRUE)
            AND (days_from_active_credit_card_disbursement_date > 180))
        THEN
            CASE
                WHEN
                    (percentage_outstanding_credit_card_active > 0.7)
                THEN
                    "4B"
                ELSE
                    "4"
            END
        WHEN
            (((active_limit_status = "High")
            OR (active_limit_status = "Medium"))
            AND (okay_cc = TRUE))
            OR
            ((active_limit_status = "Low")
            AND (okay_cc = TRUE)
            AND (days_from_active_credit_card_disbursement_date > 730))
        THEN
            CASE
                WHEN
                    (percentage_outstanding_credit_card_active > 0.7)
                THEN
                    "3B"
                ELSE
                    "3"
            END
        WHEN
            (((settled_paylater = TRUE) OR (high_paylater = TRUE))
            AND (okay = TRUE))
        THEN
            "2"
        ELSE
            "1"
    END as rule_pefindo
FROM
    pefindo_classification
),

raw_pusdafil_submit AS
  (SELECT *,
          CASE 
          WHEN loan_status = 'L' and outstanding_amount > 0 THEN 'wrong_data' 
          ELSE 'ok' 
          END AS filter_data,
          
          CASE 
          WHEN (last_day_past_due = 0 AND max_day_past_due = 90) OR 
               (last_day_past_due = 30 AND max_day_past_due = 30) THEN 'wrong_data'
          ELSE 'ok'
          END AS filter_data_2,
          
          CASE 
          WHEN max_day_past_due > TIMESTAMP_DIFF(submitDate,next_due_date, DAY) 
                THEN TIMESTAMP_DIFF(submitDate,next_due_date, DAY)
          ELSE max_day_past_due END AS calculated_max_dpd_final,
          
          TIMESTAMP_DIFF(submitDate, aggreement_date, DAY) AS delta_time,
          TIMESTAMP_DIFF(submitDate, disbursement_date, DAY) AS delta_time2,
          TIMESTAMP_DIFF(res.next_due_date, res.aggreement_date, DAY) AS tenure_from_aggreement,
          TIMESTAMP_DIFF(res.next_due_date, res.disbursement_date, DAY) AS tenure_from_disburse,
          TIMESTAMP_DIFF(res.disbursement_date, res.aggreement_date, DAY) AS time_to_disburse
   FROM (
         SELECT app.orderId,
                app.submitDate,
                inquiry.id AS inquiry_id,
                inquiry_date,
                inquiry.created_at AS inquiry_created_at,
                row_number() OVER (PARTITION BY app.orderId ORDER BY inquiry.created_at DESC) AS rank
         FROM data-platform-indodana.vayu.indodana_athena_applications app                       
         LEFT JOIN data-platform-indodana.vayu.indodana_bureau_pusdafil_inquiries inquiry 
         ON app.applicantIdNumber = inquiry.id_number 
         AND TIMESTAMP_ADD(app.submitDate, INTERVAL 3 DAY) >= inquiry.created_at
        ) 
   LEFT JOIN data-platform-indodana.vayu.indodana_bureau_pusdafil_inquiry_results res ON inquiry_id = res.pusdafil_inquiry_id
   WHERE rank = 1 
   AND TIMESTAMP_TRUNC(submitDate, DAY) > TIMESTAMP_TRUNC(disbursement_date, DAY)),
         
age_of_relationship_by_orgid AS
  (SELECT orderId,
          organizer_id,
          MIN(TIMESTAMP_DIFF(submitdate,disbursement_date,DAY)) AS latest_contract_per_orgid,
          MAX(TIMESTAMP_DIFF(submitdate,disbursement_date,DAY)) AS first_contract_per_orgid
   FROM raw_pusdafil_submit
   GROUP BY 1,
            2),

generate_feature_loyalty AS
  (SELECT orderId,
          AVG(first_contract_per_orgid - latest_contract_per_orgid) AS avg_latest_newest_age_of_relationship
   FROM age_of_relationship_by_orgid
   GROUP BY 1),

get_feature_organizer_id_level AS
  (SELECT orderId,
          organizer_id,
          SUM(CASE
                  WHEN calculated_max_dpd_final > 3
                       AND next_due_date <= submitDate
                       AND filter_data = 'ok'
                       AND filter_data_2 = 'ok' THEN 1
                  ELSE 0
              END)/
           SUM(CASE
                   WHEN next_due_date <= submitDate
                        AND filter_data = 'ok'
                        AND filter_data_2 = 'ok' THEN 1
            END) AS perc_late_3_filter,
          
          CASE
              WHEN (AVG(amount_disbursed) <= 400000)
                   OR ((APPROX_QUANTILES((TIMESTAMP_DIFF(disbursement_date, prev_disbursement_date, DAY)), 100)[OFFSET(50)] < 5
                        OR APPROX_QUANTILES(amount_disbursed, 100)[OFFSET(25)] <= 400000
                        OR APPROX_QUANTILES((TIMESTAMP_DIFF(disbursement_date, next_due_date, DAY)), 100)[OFFSET(50)] >= 180)
                       AND COUNT(*) >= 5) THEN 1
              ELSE 0
          END AS paylater_category
   FROM
     (SELECT *,
             LAG(disbursement_date, 1) OVER (PARTITION BY orderId,
                                                          organizer_id
                                             ORDER BY disbursement_date ASC) AS prev_disbursement_date
      FROM raw_pusdafil_submit)
   GROUP BY 1,
            2),

delinquency_data AS (
    SELECT raw.orderId,
    MIN(CASE WHEN filter_data = 'ok' AND filter_data_2 = 'ok' AND delta_time < 360
        AND calculated_max_dpd_final >= 30 THEN disbursement_date ELSE NULL END) as earliest_delinquency_1y,
    MIN(CASE WHEN filter_data = 'ok' AND filter_data_2 = 'ok' AND delta_time < 720
        AND calculated_max_dpd_final >= 30 THEN disbursement_date ELSE NULL END) as earliest_delinquency_2y,
    MAX(CASE WHEN filter_data = 'ok' AND filter_data_2 = 'ok' AND delta_time < 360
        AND calculated_max_dpd_final >= 30 THEN disbursement_date ELSE NULL END) as latest_delinquency_1y,
    MAX(CASE WHEN filter_data = 'ok' AND filter_data_2 = 'ok' AND delta_time < 720
        AND calculated_max_dpd_final >= 30 THEN disbursement_date ELSE NULL END) as latest_delinquency_2y
    FROM raw_pusdafil_submit raw
    GROUP BY 1
),

aggregate_function AS 
  (SELECT raw.orderId,
          COUNT(*) AS count_contract_submit,
          COUNT(CASE WHEN next_due_date <= submitDate THEN disbursement_date ELSE NULL END) AS count_contracts_submit_ndd,
          COUNT(CASE WHEN filter_data = 'ok' THEN disbursement_date ELSE NULL END) AS count_contracts_filter,
          COUNT(CASE WHEN filter_data = 'ok' AND next_due_date <= submitDate THEN disbursement_date ELSE NULL END) AS count_contracts_submit_filter_ndd,
          COUNT(CASE WHEN filter_data = 'ok' AND filter_data_2 = 'ok' then disbursement_date ELSE NULL END) AS count_contracts_submit_filter_2,
          COUNT(CASE WHEN filter_data = 'ok' AND filter_data_2 = 'ok' AND next_due_date <= submitDate THEN disbursement_date ELSE NULL END) AS count_contracts_submit_filter_2_ndd,
          COUNT(CASE WHEN filter_data = 'ok' AND filter_data_2 = 'ok' AND next_due_date <= submitDate AND TIMESTAMP_DIFF(TIMESTAMP_ADD(submitdate, INTERVAL 3 DAY), disbursement_date, DAY) <= 360 THEN disbursement_date ELSE NULL END) AS count_contracts_submit_filter_3_ndd,
          COUNT(CASE WHEN filter_data = 'ok' AND filter_data_2 = 'ok' AND next_due_date <= submitDate AND TIMESTAMP_DIFF(TIMESTAMP_ADD(submitdate, INTERVAL 3 DAY), disbursement_date, DAY) <= 270 THEN disbursement_date ELSE NULL END) AS count_contracts_submit_filter_4_ndd,
          COUNT(CASE WHEN filter_data = 'ok' AND filter_data_2 = 'ok' AND delta_time < 360 THEN disbursement_date ELSE NULL END) AS count_contracts_submit_filter_2_l12m,
          COUNT(CASE WHEN filter_data = 'ok' AND filter_data_2 = 'ok' AND delta_time < 270 THEN disbursement_date ELSE NULL END) AS count_contracts_submit_filter_3_l12m,
          COUNT(CASE WHEN filter_data = 'ok' AND filter_data_2 = 'ok' AND delta_time < 360 AND next_due_date <= submitDate THEN disbursement_date ELSE NULL END) AS count_contracts_submit_filter_2_l12m_ndd,
          COUNT(CASE WHEN paylater_category = 1 AND next_due_date <= submitDate THEN disbursement_date END) AS count_contract_submit_paylater,
          COUNT(CASE WHEN paylater_category = 1 AND filter_data = 'ok' AND filter_data_2 = 'ok' AND next_due_date <= submitDate THEN disbursement_date ELSE NULL END) AS count_contracts_submit_filter_paylater,
          SUM(CASE WHEN calculated_max_dpd_final >= 3 AND filter_data = 'ok' AND filter_data_2 = 'ok' THEN 1 ELSE 0 END) AS count_late_contract_final_3_submit_filter_2,
          SUM(CASE WHEN calculated_max_dpd_final >= 3 AND filter_data = 'ok' AND filter_data_2 = 'ok' AND next_due_date <= submitDate THEN 1 ELSE 0 END) AS count_late_contract_final_3_submit_filter_2_ndd,
          SUM(CASE WHEN calculated_max_dpd_final >= 3 AND filter_data = 'ok' AND filter_data_2 = 'ok' AND TIMESTAMP_DIFF(TIMESTAMP_ADD(submitdate, INTERVAL 3 DAY), disbursement_date, DAY) <= 360 AND next_due_date <= submitDate THEN 1 ELSE 0 END) AS count_late_contract_final_3_submit_filter_3_ndd,
          SUM(CASE WHEN calculated_max_dpd_final >= 15 AND filter_data = 'ok' AND filter_data_2 = 'ok' THEN 1 ELSE 0 END) AS count_late_contract_final_15_submit_filter_2,
          SUM(CASE WHEN calculated_max_dpd_final >= 15 AND filter_data = 'ok' AND filter_data_2 = 'ok' AND next_due_date <= submitDate THEN 1 ELSE 0 END) AS count_late_contract_final_15_submit_filter_2_ndd,
          SUM(CASE WHEN calculated_max_dpd_final >= 15 AND filter_data = 'ok' AND filter_data_2 = 'ok'AND TIMESTAMP_DIFF(TIMESTAMP_ADD(submitdate, INTERVAL 3 DAY), disbursement_date, DAY) <= 360 AND next_due_date <= submitDate THEN 1 ELSE 0 END) AS count_late_contract_final_15_submit_filter_3_ndd,
          SUM(CASE WHEN calculated_max_dpd_final >= 90 AND filter_data = 'ok' AND filter_data_2 = 'ok' THEN 1 ELSE 0 END) AS count_late_contract_final_90_submit_filter_2,
          SUM(CASE WHEN calculated_max_dpd_final >= 90 AND filter_data = 'ok' AND filter_data_2 = 'ok' AND next_due_date <= submitDate THEN 1 ELSE 0 END) AS count_late_contract_final_90_submit_filter_2_ndd,
          SUM(CASE WHEN calculated_max_dpd_final >= 90 AND filter_data = 'ok' AND filter_data_2 = 'ok'AND TIMESTAMP_DIFF(TIMESTAMP_ADD(submitdate, INTERVAL 3 DAY), disbursement_date, DAY) <= 360 AND next_due_date <= submitDate THEN 1 ELSE 0 END) AS count_late_contract_final_90_submit_filter_3_ndd,
          SUM(CASE WHEN amount_disbursed <= 500000 THEN 1 ELSE 0 END) AS disburse_less_than,
          SUM(CASE WHEN amount_disbursed > 500000 THEN 1 ELSE 0 END) AS disburse_more_than,
          SUM(CASE WHEN amount_disbursed <= 500000 THEN 1 ELSE NULL END) AS count_disbursed_less_than_500000,
          SUM(CASE WHEN amount_disbursed > 500000 THEN 1 ELSE NULL END) AS count_disbursed_more_than_500000,
          SUM(CASE WHEN next_due_date <= submitDate AND amount_disbursed <= 500000 THEN 1 ELSE 0 END) AS disburse_less_than_ndd,
          SUM(CASE WHEN next_due_date <= submitDate AND amount_disbursed > 500000 THEN 1 ELSE 0 END) AS disburse_more_than_ndd,
          SUM(CASE WHEN filter_data = 'ok' AND filter_data_2 = 'ok' AND outstanding_amount > 0 AND (TIMESTAMP_DIFF(submitDate, aggreement_date, DAY) <= 360 OR TIMESTAMP_DIFF(submitDate, disbursement_date, DAY) <= 360) AND last_day_past_due > 0 AND last_day_past_due < 30 THEN outstanding_amount * (last_day_past_due/30)
                   WHEN filter_data = 'ok' AND filter_data_2 = 'ok' AND outstanding_amount > 0 AND (TIMESTAMP_DIFF(submitDate, aggreement_date, DAY) <= 360 OR TIMESTAMP_DIFF(submitDate, disbursement_date, DAY) <= 360) AND last_day_past_due >= 30 THEN outstanding_amount
              END) AS discounted_sum_outstanding_late_1_year_filter_2,
          SUM(CASE WHEN filter_data = 'ok' AND filter_data_2 = 'ok' AND outstanding_amount > 0 AND (TIMESTAMP_DIFF(submitDate, aggreement_date, DAY) <= 270 OR TIMESTAMP_DIFF(submitDate, disbursement_date, DAY) <= 270) AND last_day_past_due > 0 AND last_day_past_due < 30 THEN outstanding_amount * (last_day_past_due/30)
                   WHEN filter_data = 'ok' AND filter_data_2 = 'ok' AND outstanding_amount > 0 AND (TIMESTAMP_DIFF(submitDate, aggreement_date, DAY) <= 270 OR TIMESTAMP_DIFF(submitDate, disbursement_date, DAY) <= 270) AND last_day_past_due >= 30 THEN outstanding_amount
              END) AS discounted_sum_outstanding_late_9_mo_filter_3,
          SUM(CASE WHEN filter_data = 'ok' AND filter_data_2 = 'ok' AND next_due_date <= submitDate AND outstanding_amount > 0 AND (TIMESTAMP_DIFF(submitDate, aggreement_date, DAY) <= 360 OR TIMESTAMP_DIFF(submitDate, disbursement_date, DAY) <= 360) AND last_day_past_due > 0 AND last_day_past_due < 30 THEN outstanding_amount * (last_day_past_due/30)
                   WHEN filter_data = 'ok' AND filter_data_2 = 'ok' AND next_due_date <= submitDate AND outstanding_amount > 0 AND (TIMESTAMP_DIFF(submitDate, aggreement_date, DAY) <= 360 OR TIMESTAMP_DIFF(submitDate, disbursement_date, DAY) <= 360) AND last_day_past_due >= 30 THEN outstanding_amount
              END) AS discounted_sum_outstanding_late_1_year_filter_2_ndd,
          AVG(amount_disbursed) AS avg_amount_disbursed,
          SUM(CASE WHEN paylater_category = 1 AND calculated_max_dpd_final >= 3 AND filter_data = 'ok' AND filter_data_2 = 'ok' AND next_due_date <= submitDate THEN 1 ELSE 0 END) AS count_late_contract_final_3_submit_filter_paylater,
          COUNT(DISTINCT CASE WHEN perc_late_3_filter > 0.2 THEN orgid.organizer_id END) AS count_org_id_late3_morethan_20_percent_filter,
          
          COUNT(DISTINCT CASE 
          WHEN 
               filter_data = 'ok' 
               AND filter_data_2 = 'ok' 
               AND max_day_past_due >= 90 
           THEN  raw.organizer_id ELSE NULL END) AS fdc_count_dpd_90d,
           
          COUNT(DISTINCT CASE 
          WHEN 
               filter_data = 'ok' 
               AND filter_data_2 = 'ok' 
               AND max_day_past_due >= 90 
               AND TIMESTAMP_DIFF(TIMESTAMP_ADD(submitdate, INTERVAL 3 DAY), disbursement_date, DAY) <= 360 
          THEN raw.organizer_id ELSE NULL END) AS fdc_count_dpd_90d_1y,
          
          COUNT(DISTINCT CASE 
          WHEN 
               filter_data = 'ok' 
               AND filter_data_2 = 'ok' 
               AND max_day_past_due >= 15
               AND TIMESTAMP_DIFF(TIMESTAMP_ADD(submitdate, INTERVAL 3 DAY), disbursement_date, DAY) <= 180 
          THEN raw.organizer_id ELSE NULL END) AS fdc_count_dpd_15d_6mo,

          MAX(CASE 
          WHEN 
               filter_data = 'ok' 
               AND filter_data_2 = 'ok' 
               AND amount_disbursed > 100000
               AND TIMESTAMP_DIFF(TIMESTAMP_ADD(submitdate, INTERVAL 3 DAY), disbursement_date, DAY) <= 360 
           THEN calculated_max_dpd_final ELSE NULL END) AS fdc_max_dpd_above_100k_1y,
           
           MAX(CASE 
           WHEN 
               filter_data = 'ok' 
               AND filter_data_2 = 'ok' 
               AND amount_disbursed > 100000
               AND TIMESTAMP_DIFF(TIMESTAMP_ADD(submitdate, INTERVAL 3 DAY), disbursement_date, DAY) <= 270 
           THEN calculated_max_dpd_final ELSE NULL END) AS fdc_max_dpd_above_100k_9mo,
           
           COUNT(DISTINCT CASE
           WHEN filter_data = 'ok' 
                AND filter_data_2 = 'ok' 
                AND TIMESTAMP_DIFF(TIMESTAMP_ADD(submitdate, INTERVAL 3 DAY), disbursement_date, DAY) <= 360
                AND loan_status = 'L'
                AND outstanding_amount = 0
          THEN format_timestamp('%Y-%m',disbursement_date) ELSE NULL END) AS fdc_count_unique_month_paid_1y,
          
                     COUNT(DISTINCT CASE
           WHEN filter_data = 'ok' 
                AND filter_data_2 = 'ok' 
                AND TIMESTAMP_DIFF(TIMESTAMP_ADD(submitdate, INTERVAL 3 DAY), disbursement_date, DAY) <= 270
                AND loan_status = 'L'
                AND outstanding_amount = 0
          THEN format_timestamp('%Y-%m',disbursement_date) ELSE NULL END) AS fdc_count_unique_month_paid_9mo,
          
    COUNT(DISTINCT CASE WHEN filter_data = 'ok' AND filter_data_2 = 'ok' 
        AND TIMESTAMP_DIFF(disbursement_date,latest_delinquency_2y, DAY) > 0 THEN raw.organizer_id ELSE NULL END)
        AS count_unique_since_delinquency_2y,
    
    COUNT(DISTINCT CASE WHEN filter_data = 'ok' AND filter_data_2 = 'ok' 
        AND loan_status = 'L' AND outstanding_amount = 0 AND calculated_max_dpd_final < 30
        AND TIMESTAMP_DIFF(disbursement_date,latest_delinquency_2y, DAY) > 0
        THEN FORMAT_TIMESTAMP("%Y-%m",disbursement_date) ELSE NULL END)
        AS count_unique_month_paid_since_max_delinquency_2y
          
   FROM raw_pusdafil_submit AS raw
   LEFT JOIN get_feature_organizer_id_level AS orgid ON raw.orderId = orgid.orderId
   AND raw.organizer_id = orgid.organizer_id
   LEFT JOIN delinquency_data dd ON raw.orderId = dd.orderId
   GROUP BY 1),

    fdc as (
        SELECT ra.orderId,count_contract_submit,
          CASE 
          WHEN count_contract_submit IS NULL THEN -99 
          WHEN (count_contracts_submit_filter_2_ndd IS NULL OR count_contracts_submit_filter_2_ndd = 0) THEN -99 
          ELSE count_late_contract_final_3_submit_filter_2_ndd/count_contracts_submit_filter_2_ndd 
          END AS percentage_of_calculated_max_day_past_due_3_contract_due_date_before_submit,

          CASE 
          WHEN count_contract_submit IS NULL THEN -99
          WHEN (count_contracts_submit_filter_2_ndd IS NULL OR count_contracts_submit_filter_2_ndd = 0) THEN -99 
          ELSE count_late_contract_final_15_submit_filter_2_ndd/count_contracts_submit_filter_2_ndd 
          END AS percentage_of_calculated_max_day_past_due_15_contract_due_date_before_submit,
          
          CASE 
          WHEN count_contract_submit IS NULL THEN -99
          WHEN (count_contracts_submit_filter_2_ndd IS NULL OR count_contracts_submit_filter_2_ndd = 0) THEN -99 
          ELSE count_late_contract_final_90_submit_filter_2_ndd/count_contracts_submit_filter_2_ndd 
          END AS percentage_of_calculated_max_day_past_due_90_contract_due_date_before_submit,
          
          CASE 
          WHEN count_contract_submit IS NULL OR count_contract_submit = 0 THEN -99 
          ELSE (disburse_less_than/count_contract_submit)*disburse_less_than END AS weighted_count_paylater,
          
          CASE 
          WHEN count_contract_submit IS NULL THEN -99 
          WHEN (count_contracts_submit_filter_2 IS NULL OR count_contracts_submit_filter_2 = 0 OR count_contracts_submit_filter_2_l12m IS NULL OR count_contracts_submit_filter_2_l12m = 0) THEN -99
          WHEN discounted_sum_outstanding_late_1_year_filter_2 IS NULL THEN 0
          ELSE discounted_sum_outstanding_late_1_year_filter_2
          END AS discounted_sum_outstanding_late_1y,
          
          CASE 
          WHEN count_contract_submit IS NULL THEN -99 
          WHEN (count_contracts_submit_filter_2 IS NULL OR count_contracts_submit_filter_2 = 0 OR count_contracts_submit_filter_3_l12m IS NULL OR count_contracts_submit_filter_3_l12m = 0) THEN -99
          WHEN discounted_sum_outstanding_late_9_mo_filter_3 IS NULL THEN 0
          ELSE discounted_sum_outstanding_late_9_mo_filter_3
          END AS discounted_sum_outstanding_late_9mo,
          
          CASE
          WHEN count_contract_submit is null OR count_contract_submit = 0 THEN -999
          ELSE avg_amount_disbursed  END AS avg_amount_disbursed_total,
          
          CASE 
          WHEN count_contract_submit is null OR count_contract_submit = 0 THEN '-999'
          WHEN (count_disbursed_more_than_500000 IS NULL AND count_disbursed_less_than_500000 <= 10) OR
               ((count_disbursed_less_than_500000 IS NOT NULL AND count_disbursed_more_than_500000 IS NOT NULL) AND 
               (count_disbursed_less_than_500000/count_disbursed_more_than_500000) > 1 AND count_disbursed_less_than_500000 <= 10) THEN 'low_paylater'
          WHEN (count_disbursed_more_than_500000 IS NULL AND count_disbursed_less_than_500000 > 10) OR
               ((count_disbursed_less_than_500000 IS NOT NULL AND count_disbursed_more_than_500000 IS NOT NULL) AND 
               (count_disbursed_less_than_500000/count_disbursed_more_than_500000) > 1 AND count_disbursed_less_than_500000 > 10)THEN 'high_paylater'
          WHEN (count_disbursed_less_than_500000 IS NULL AND count_disbursed_more_than_500000 <= 10) OR
               ((count_disbursed_less_than_500000 IS NOT NULL AND count_disbursed_more_than_500000 IS NOT NULL) AND 
                (count_disbursed_less_than_500000/count_disbursed_more_than_500000) <= 1 AND count_disbursed_more_than_500000 <= 10)THEN 'low_cashloan'
          WHEN (count_disbursed_less_than_500000 IS NULL AND count_disbursed_more_than_500000 > 10) OR
               ((count_disbursed_less_than_500000 IS NOT NULL AND count_disbursed_more_than_500000 IS NOT NULL) AND 
               (count_disbursed_less_than_500000/count_disbursed_more_than_500000) <= 1 AND count_disbursed_more_than_500000 > 10)THEN 'high_cashloan'
         ELSE 'other'
         END AS product_preference_category,
         
         CASE
         WHEN (count_contracts_submit_filter_ndd IS NULL
               OR count_contracts_submit_filter_ndd = 0) THEN -999 --don't have fdc at all (with filter next due date)
         WHEN (count_contract_submit_paylater IS NULL
               OR count_contract_submit_paylater = 0) THEN -98 --have fdc, but don't have paylater organizerid
         WHEN (count_contracts_submit_filter_paylater IS NULL
                OR count_contracts_submit_filter_paylater = 0) THEN -99 --have fdc and paylater contract, but all contracts are filtered out (by error)
         ELSE count_late_contract_final_3_submit_filter_paylater/count_contracts_submit_filter_paylater -- calculate percentage late contract in paylater organizer id
         END AS percentage_of_calculated_max_day_past_due_3_paylater_contract_due_date_before_submit, 
       
         CASE
         WHEN (count_contracts_submit_filter_ndd IS NULL
               OR count_contracts_submit_filter_ndd = 0) THEN -999 --don't have fdc at all (with filter next due date)
         WHEN (count_contracts_submit_filter_2 = 0
               OR count_contracts_submit_filter_2 IS NULL) THEN -99 --have fdc, but all contracts are filtered out (by error)
         ELSE count_org_id_late3_morethan_20_percent_filter
         END AS count_org_id_dpd_3_morethan_20_percent,
       
         CASE
         WHEN (count_contract_submit IS NULL
               OR count_contract_submit = 0) THEN -999 --don't have fdc at all
         ELSE avg_latest_newest_age_of_relationship
         END AS avg_latest_newest_age_of_relationship,
         
        CASE 
        WHEN IFNULL(count_contracts_submit_ndd,0) = 0 THEN -999
        WHEN IFNULL(count_contracts_submit_filter_2_ndd,0)=0  THEN -99
        WHEN fdc_count_dpd_90d IS NULL THEN 0
        ELSE fdc_count_dpd_90d END AS fdc_count_dpd_90d,
           
        CASE 
        WHEN IFNULL(count_contracts_submit_ndd,0) = 0 THEN -999
        WHEN IFNULL(count_contracts_submit_filter_3_ndd,0)=0  THEN -99
        WHEN fdc_count_dpd_90d_1y IS NULL THEN 0
        ELSE fdc_count_dpd_90d_1y END AS fdc_count_dpd_90d_1y,
        
        CASE 
        WHEN IFNULL(count_contracts_submit_ndd,0) = 0 THEN -999
        WHEN IFNULL(count_contracts_submit_filter_2_ndd,0)=0  THEN -99
        WHEN fdc_count_dpd_15d_6mo IS NULL THEN 0
        ELSE fdc_count_dpd_15d_6mo END AS fdc_count_dpd_15d_6mo,
        
        CASE 
        WHEN IFNULL(count_contracts_submit_ndd,0) = 0 THEN -999
        WHEN IFNULL(count_contracts_submit_filter_3_ndd,0)=0  THEN -99
        WHEN fdc_max_dpd_above_100k_1y < 0 THEN 0
        ELSE fdc_max_dpd_above_100k_1y END AS fdc_max_dpd_above_100k_1y,

        CASE 
        WHEN IFNULL(count_contracts_submit_ndd,0) = 0 THEN -999
        WHEN IFNULL(count_contracts_submit_filter_4_ndd,0)=0  THEN -99
        WHEN fdc_max_dpd_above_100k_9mo < 0 THEN 0
        ELSE fdc_max_dpd_above_100k_9mo END AS fdc_max_dpd_above_100k_9mo,
       
        CASE 
        WHEN IFNULL(count_contracts_submit_ndd,0) = 0 THEN -999
        WHEN IFNULL(count_contracts_submit_filter_3_ndd,0)=0 THEN -99
        ELSE fdc_count_unique_month_paid_1y END AS fdc_count_unique_month_paid_1y,
        
        CASE 
        WHEN IFNULL(count_contracts_submit_ndd,0) = 0 THEN -999
        WHEN IFNULL(count_contracts_submit_filter_4_ndd,0)=0 THEN -99
        ELSE fdc_count_unique_month_paid_9mo END AS fdc_count_unique_month_paid_9mo,
        
        CASE 
        WHEN IFNULL(count_contracts_submit_ndd,0) = 0 THEN -999
        WHEN (count_contracts_submit_filter_3_ndd IS NULL OR count_contracts_submit_filter_3_ndd = 0) THEN -99 
        ELSE count_late_contract_final_3_submit_filter_3_ndd/count_contracts_submit_filter_3_ndd 
        END AS percentage_of_calculated_max_day_past_due_3_contract_due_date_before_submit_1y,

        CASE 
        WHEN IFNULL(count_contracts_submit_ndd,0) = 0 THEN -999
        WHEN (count_contracts_submit_filter_3_ndd IS NULL OR count_contracts_submit_filter_3_ndd = 0) THEN -99 
        ELSE count_late_contract_final_15_submit_filter_3_ndd/count_contracts_submit_filter_3_ndd 
        END AS percentage_of_calculated_max_day_past_due_15_contract_due_date_before_submit_1y,
        
        CASE 
        WHEN IFNULL(count_contracts_submit_ndd,0) = 0 THEN -99 
        WHEN (count_contracts_submit_filter_3_ndd IS NULL OR count_contracts_submit_filter_3_ndd = 0) THEN -99 
        ELSE count_late_contract_final_90_submit_filter_3_ndd/count_contracts_submit_filter_3_ndd END 
        AS percentage_of_calculated_max_day_past_due_90_contract_due_date_before_submit_1y,

        
        CASE 
        WHEN IFNULL(count_contracts_submit_ndd,0) = 0 THEN -999 
        WHEN latest_delinquency_2y IS NULL THEN -99 
        WHEN IFNULL(count_unique_since_delinquency_2y,0) = 0 THEN -98
        ELSE IFNULL(count_unique_month_paid_since_max_delinquency_2y,0)
        END AS count_unique_month_paid_since_max_delinquency_2y
         
   FROM `athena-179008.vayu_data_mart.dm_application_risk_analytics` ra
   LEFT JOIN aggregate_function agg ON ra.orderId = agg.orderId
   LEFT JOIN generate_feature_loyalty AS loyalty ON loyalty.orderId = ra.orderId
   LEFT JOIN delinquency_data dd ON ra.orderId = dd.orderId),

fraud_rule_temp as (
select orderId, tiket_score,izi_max_multi_inquiries_90d,izi_max_multi_inquiries_14d,number_of_related_previous_cashloan_order_id,avg_amount_disbursed_total,bpjstk_status_klaim,case
when (phone_age_ever_name_match >= 5 or email_age_ever_name_match >=5 or pefindo_individuals_phone_number_match > 0 or pefindo_info_histories_phone_number_match > 0 or djp_email_match > 0)
    AND (verified_izi_mobile_phone_whatsapp = 'verified' or izi_mobile_phone_number_ages = '10-12month' or izi_mobile_phone_number_ages = '12month+' or pefindo_individuals_phone_number_match > 0 or pefindo_info_histories_phone_number_match > 0)
    then 'not fraud' else 'fraud candidate' end as fraud_rule_temp_1, blibli_score, blibli_account_age_in_days
from (
SELECT app.userId,
       app.orderid,    
    CAST(feat4.value as NUMERIC) as izi_max_multi_inquiries_90d,
    CAST(feat41.value as NUMERIC) as izi_max_multi_inquiries_14d,
    CAST(feat9.value as NUMERIC) as phone_age_ever_name_match,
    CAST(feat10.value as NUMERIC) as email_age_ever_name_match,
    feat11.value as verified_izi_mobile_phone_whatsapp,
    feat12.value as izi_mobile_phone_number_ages,
    CAST(feat13.value as NUMERIC) as djp_email_match,
    CAST(feat14.value as NUMERIC) as pefindo_individuals_phone_number_match,
    CAST(feat15.value as NUMERIC) as pefindo_info_histories_phone_number_match,
    CAST(feat16.value as NUMERIC) as no_email_dtlab_match_email,
    CAST(feat17.value as NUMERIC) as tiket_score,
    CAST(feat18.value as NUMERIC) as number_of_related_previous_cashloan_order_id,
    CAST(feat19.value as NUMERIC) as avg_amount_disbursed_total,
    feat20.value as bpjstk_status_klaim,
    CAST(feat21.value as NUMERIC) as blibli_score,
    CAST(feat22.value as NUMERIC) as blibli_account_age_in_days


FROM athena-179008.vayu_data_mart.dm_application_risk_analytics app
LEFT JOIN data-platform-indodana.vayu.indodana_featurestores_object_features_scores feat4 on feat4.object_id = app.orderId and feat4.feature_slug_name = 'izi_max_multi_inquiries_90d'
LEFT JOIN data-platform-indodana.vayu.indodana_featurestores_object_features_scores feat41 on feat41.object_id = app.orderId and feat41.feature_slug_name = 'izi_max_multi_inquiries_14d'
LEFT JOIN data-platform-indodana.vayu.indodana_featurestores_object_features_scores feat9 on feat9.object_id = app.orderId and feat9.feature_slug_name = 'phone_age_ever_name_match'
LEFT JOIN data-platform-indodana.vayu.indodana_featurestores_object_features_scores feat10 on feat10.object_id = app.orderId and feat10.feature_slug_name = 'email_age_ever_name_match'
LEFT JOIN data-platform-indodana.vayu.indodana_featurestores_object_features_scores feat11 on feat11.object_id = app.orderId and feat11.feature_slug_name = 'verified_izi_mobile_phone_whatsapp'
LEFT JOIN data-platform-indodana.vayu.indodana_featurestores_object_features_scores feat12 on feat12.object_id = app.orderId and feat12.feature_slug_name = 'izi_mobile_phone_number_ages'

LEFT JOIN data-platform-indodana.vayu.indodana_featurestores_object_features_scores feat13 on feat13.object_id = app.orderId and feat13.feature_slug_name = 'djp_email_match'
LEFT JOIN data-platform-indodana.vayu.indodana_featurestores_object_features_scores feat14 on feat14.object_id = app.orderId and feat14.feature_slug_name = 'pefindo_individuals_phone_number_match'
LEFT JOIN data-platform-indodana.vayu.indodana_featurestores_object_features_scores feat15 on feat15.object_id = app.orderId and feat15.feature_slug_name = 'pefindo_info_histories_phone_number_match'
LEFT JOIN data-platform-indodana.vayu.indodana_featurestores_object_features_scores feat16 on app.orderId = feat16.object_id and feat16.feature_slug_name = 'no_email_dtlab_match_email'
LEFT JOIN data-platform-indodana.vayu.indodana_featurestores_object_features_scores feat17 on app.orderId = feat17.object_id and feat17.feature_slug_name = 'tiket_score'
LEFT JOIN data-platform-indodana.vayu.indodana_featurestores_object_features_scores feat18 on app.orderId = feat18.object_id and feat18.feature_slug_name = 'number_of_related_previous_cashloan_order_id'
LEFT JOIN data-platform-indodana.vayu.indodana_featurestores_object_features_scores feat19 on app.orderId = feat19.object_id and feat19.feature_slug_name = 'avg_amount_disbursed_total'
LEFT JOIN data-platform-indodana.vayu.indodana_featurestores_object_features_scores feat20 on app.orderId = feat20.object_id and feat19.feature_slug_name = 'bpjstk_status_klaim'

LEFT JOIN data-platform-indodana.vayu.indodana_featurestores_object_features_scores feat21 on app.orderId = feat21.object_id and feat21.feature_slug_name = 'blibli_score'
LEFT JOIN data-platform-indodana.vayu.indodana_featurestores_object_features_scores feat22 on app.orderId = feat22.object_id and feat22.feature_slug_name = 'blibli_account_age_in_days'


)
),
    
    
data as (
SELECT app.userId,
       app.orderid,
       app2.status,
       app.isRepeatOrder,
       app.application_status,
       app.requested_loan_amount,
       app.requested_loan_tenure,
       app.product_type,
       app.customersegment,
       app.customersegment_BPJS,
       app.submitDate,
       app.submitdateyear,
       app.submitdatemonth,
       app.submitdateweek,
       app.approveddate,
       app.approveddateyear,
       app.approveddatemonth,
       app.approveddateweek,
       app.approvedLoanAmount,
       app.approvedLoanTenureInMonth,
       app.contract_status,
       app.dayPastDue,
       app.maxDayPastDue,
       app.dpd_installment_1,
       app.dpd_installment_2,
       app.dpd_installment_3,
       app.dpd_installment_4,
       app.dpd_installment_5,
       app.dpd_installment_6,
       app.cntpayment,
       app.applicant_gender,
       app.applicant_stated_work_city,
       app.applicant_stated_monthly_income,
       app.CompanyCityGroupBreakdown,
       app.typeOfJobs,
       app.reject_reason_ranked,
       app.deviation_ranked,
       app.latest_model_score_model_norescore,
       app.latest_model_score_norescore,
        CASE 
        WHEN app.product_type = 'CASH_LOAN' and app.isRepeatOrder = FALSE THEN 'CASH_LOAN_NEW'
        WHEN app.product_type = 'CASH_LOAN' and app.isRepeatOrder = TRUE THEN 'CASH_LOAN_RO'
        WHEN app.product_type = 'CREDIT_LIMIT' and app.partner is NULL THEN 'CREDIT_LIMIT_INDODANA'
        WHEN app.product_type = 'CREDIT_LIMIT' and app.partner is NOT NULL THEN CONCAT('CREDIT_LIMIT_',app.partner)
        ELSE app.product_type END AS business_segment,
        CASE WHEN UPPER(app2.currentCompanyIndustry) IN ('ASURANSI/FOREX','HUKUM DAN PERPAJAKAN','KESEHATAN/KLINIK','KEUANGAN/BANK','KONSULTAN','PEMERINTAHAN','PERKEBUNAN','PERTAMBANGAN','SPESIALIS','TELEKOMUNIKASI','INDUSTRI/PABRIK','KONTRAKTOR/PROPERTI') THEN 'low_risk_industry'
            WHEN UPPER(app2.currentCompanyIndustry) IN ('JASA/PELAYANAN','PERDAGANGAN','RETAIL') THEN 'medium_risk_industry' ELSE 'high_risk_industry' END AS industry_segment,

       app.flag_dpd_15, app.flag_dpd_30,app.flag_dpd_3_installment_1 as flag_dpd_3,app.flag_dpd_1,
       app.flag_dpd_15_installment_1,
       app.flag_dpd_7_installment_1,
       app.flag_dpd_30_installment_1,
       app.flag_dpd_15_installment_2,
       app.flag_dpd_30_installment_2,
       app.flag_dpd_15_installment_3,
       app.flag_dpd_30_installment_3,
       app.flag_dpd_60,app.flag_dpd_90, app.outstanding_principal, app.partner,
       app.`limit`,app.limit_balance, 
       case 
       when app.bucket_category = 'BUCKET_INDODANA_1330' then app.`limit` * 3
       when app.bucket_category = 'BUCKET_INDODANA_1350' then app.`limit` * 5
       else app.`limit` end as limit_exposure,
       app.approveddate_month, app.submitdate_month,app.nth_approved_app_CLI,
    app.pefindo_score, app.pefindo_model, app.identity_score, app.identity_model,app.auto_approve_result, app.auto_approve_pefindo_category, app.auto_approve_tag,
    case when app.submitDate >= '2020-12-09' then 'new policy' else 'old policy' end as policy_change_period,
    fdc.count_contract_submit,fdc.fdc_count_unique_month_paid_1y,
    pefindo_rule.rule_pefindo as rule_pefindo_query,
    identity_score_mapping.identity_score_manual,
    fraud_rule_temp.fraud_rule_temp_1, fraud_rule_temp.tiket_score, fraud_rule_temp.bpjstk_status_klaim,
    fraud_rule_temp.blibli_score, fraud_rule_temp.blibli_account_age_in_days,
    case 
    when app.partner = 'BLIBLI' and fraud_rule_temp.blibli_score < 0 OR fraud_rule_temp.blibli_account_age_in_days < 0 then 'new blibli' 
    when app.partner = 'BLIBLI' and (fraud_rule_temp.blibli_score >= 0 AND fraud_rule_temp.blibli_account_age_in_days >= 0) then 'existing blibli' 
    when app.partner = 'TIKET' and fraud_rule_temp.tiket_score < 0 then 'new tiket' 
    when app.partner = 'TIKET' and fraud_rule_temp.tiket_score >= 0 then 'existing tiket' 
    else 'normal' end as user_category,
    fraud_rule_temp.number_of_related_previous_cashloan_order_id,fraud_rule_temp.avg_amount_disbursed_total,
    percentage_of_calculated_max_day_past_due_3_paylater_contract_due_date_before_submit,
    discounted_sum_outstanding_late_1y,izi_max_multi_inquiries_90d,izi_max_multi_inquiries_14d,
    pefindos_agg_fetch.flag_pefindo, prot.score as score_prototype_ro, 
    noo.score as score_prototype_new, part.score as score_prototype_partner, 
    part2.score_fdc_indodana as score_prototype_partner_indodana,
    part3.score_fdc_indodana as score_prototype_partner_indodana_cli,
    JSON_EXTRACT(abTestingExperiment,'$.cli_show_income_validation_experiment') as cli_show_income_validation_experiment,
    case 
    when app2.marketing_utmMedium = 'website' then 'website experiment'
    when JSON_EXTRACT(abTestingExperiment,'$.cli_show_income_validation_experiment') = 'false' then 'hide income validation'
    when JSON_EXTRACT(abTestingExperiment,'$.cli_show_income_validation_experiment') = 'true' then 'show income validation'
    else 'normal' end as experiment
    
    , app2.createdAt,
    case when app.submitDate is not null then 'submit' else 'not submit' end as submit_flag,
    tiket_new_score.finalScore as tiket_new_score, apo.blibli_partner_feature
    
FROM athena-179008.vayu_data_mart.dm_application_risk_analytics app
left join data-platform-indodana.vayu.indodana_athena_applications app2 on app.orderId = app2.orderId
LEFT JOIN fdc on fdc.orderId = app.orderId
LEFT JOIN pefindo_rule on pefindo_rule.orderId = app.orderId
LEFT JOIN identity_score_mapping on identity_score_mapping.orderId = app.orderId
LEFT JOIN fraud_rule_temp on fraud_rule_temp.orderId = app.orderId
LEFT JOIN pefindos_agg_fetch on app2.applicantIdNumber = pefindos_agg_fetch.idNumber
LEFT JOIN (select orderId, score from athena-179008.vayu_data_mart.prototype_score_repeat_order_mar_21) prot on app.orderId = prot.orderId
LEFT JOIN athena-179008.vayu_data_mart.prototype_score_new_order_mar_21 noo on app.orderId = noo.orderId
LEFT JOIN athena-179008.vayu_data_mart.prototype_score_cli_partner_mar_21 part on app.orderId = part.orderId
LEFT JOIN athena-179008.vayu_data_mart.prototype_score_cli_partner_on_clicashloan_mar_21 part2 on app.orderId = part2.orderId
LEFT JOIN athena-179008.vayu_data_mart.prototype_score_cli_partner_on_cli_indodana_mar_21 part3 on app.orderId = part3.orderId
LEFT JOIN tiket_new_score on tiket_new_score.orderId = app.orderId
LEFT JOIN data-platform-indodana.vayu.indodana_apollo_application_assembled apo on apo.order_id = app.orderId
)
SELECT * FROM (
    select business_segment
        , user_category
        , COUNT(1) as submit
        , SUM(CASE WHEN status='Approved' THEN 1 ELSE 0 END) as approve
        , SUM(CASE WHEN status='Approved' THEN 1 ELSE 0 END)*100.00/COUNT(1) as approve_rate
        , 'All Time' as period
    from data
    where business_segment IN ('CREDIT_LIMIT_BLIBLI','CREDIT_LIMIT_INDODANA','CREDIT_LIMIT_TIKET') and submitDate is not null
    GROUP BY 1,2
    UNION ALL

    select business_segment
        , user_category
        , COUNT(1) as submit
        , SUM(CASE WHEN status='Approved' THEN 1 ELSE 0 END) as approve
        , SUM(CASE WHEN status='Approved' THEN 1 ELSE 0 END)*100.00/COUNT(1) as approve_rate
        , 'Last 30 Days' as period
    from data
    where business_segment IN ('CREDIT_LIMIT_BLIBLI','CREDIT_LIMIT_INDODANA','CREDIT_LIMIT_TIKET') and DATE(submitDate) >= DATE_ADD(CURRENT_DATE,INTERVAL -30 DAY)
    GROUP BY 1,2

    UNION ALL

    select business_segment
        , user_category
        , COUNT(1) as submit
        , SUM(CASE WHEN status='Approved' THEN 1 ELSE 0 END) as approve
        , SUM(CASE WHEN status='Approved' THEN 1 ELSE 0 END)*100.00/COUNT(1) as approve_rate
        , 'Last 3 Months' as period
    from data
    where business_segment IN ('CREDIT_LIMIT_BLIBLI','CREDIT_LIMIT_INDODANA','CREDIT_LIMIT_TIKET') and DATE(submitDate) >= DATE_ADD(CURRENT_DATE,INTERVAL -3 MONTH)
    GROUP BY 1,2
)


In [17]:
param1.shape
# param1.dtypes

(27, 6)

In [18]:
param1.head()

,business_segment,user_category,submit,approve,approve_rate,period
0,CREDIT_LIMIT_INDODANA,normal,13894,3218,23.161077,Last 3 Months
1,CREDIT_LIMIT_TIKET,new tiket,1398,471,33.690987,Last 3 Months
2,CREDIT_LIMIT_BLIBLI,normal,1037,0,0.000000,Last 3 Months
3,CREDIT_LIMIT_TIKET,new blibli,74462,21176,28.438667,Last 3 Months
4,CREDIT_LIMIT_INDODANA,new blibli,305352,32341,10.591383,Last 3 Months


Export to excel/csv

In [19]:
from datetime import date
# print(date.today())
today_str = str(date.today()).replace('-','')
filename = 'Data_Approval_Rate_Tiket_' + today_str + '.xlsx'
print(filename)

Data_Approval_Rate_Tiket_20210603.xlsx


In [20]:
param1.to_excel(filename,index=False)